Tristan J. Hillis <br/>
ID: 1260315 <br/>
Astro 425 | Homework # 4

In [2]:
%pylab inline

# Other imports
from scipy.integrate import quad
from matplotlib.ticker import AutoMinorLocator

# style changes
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10.0, 8.0)
mpl.rc('lines', linewidth=1.8, color='k')
mpl.rcParams['font.size'] = 14

def ticks(labelsize, major_size, minor_size):
    mpl.rc('xtick', labelsize=labelsize)
    mpl.rc('xtick.major', size=major_size)
    mpl.rc('xtick.minor', size=minor_size)
    mpl.rc('ytick', labelsize=labelsize)
    mpl.rc('ytick.major', size=major_size)
    mpl.rc('ytick.minor', size=minor_size)
    
ticks(15, 10.0, 5)

Populating the interactive namespace from numpy and matplotlib


In [27]:
class Cosmology():
    
    def __init__(self, omega_r, omega_m, omega_lambda):
        self.omega_r = omega_r
        self.omega_m = omega_m
        self.omega_lambda = omega_lambda
        self.omega_o = omega_r + omega_m + omega_lambda
    
         
    def H_z(self, H, omega_r, omega_m, omega_l, omega_o, z):
        return H * np.sqrt(omega_r * (1.0 + z) ** 4 + omega_m * (1.0 + z) ** 3 + omega_l + (1.0 - omega_o) * (1.0 + z) ** 2)
    
    def printModel(self):
        print "Omega_r = %.3f, Omega_m = %.3f, Omega_lambda = %.3f, Omega_o = %.3f"%(self.omega_r, self.omega_m, self.omega_lambda, self.omega_o)
    
    def comovingFrom0toZ(self, zmax, resolution=1000.0):
        H = 2.2683e-18 # hubble constant in s^-1
        z_range = np.linspace(0, zmax, resolution)
        model = lambda z: 299792458.0 / self.H_z(H, self.omega_r, self.omega_m, self.omega_lambda, self.omega_o, z)
        results = np.asarray([quad(model, 0, i)[0] for i in z_range])
        return [z_range, results * 3.24e-23] # return results in Mpc
    
    def timeFrom0toZ(self, zmax, resolution=1000.0):
        H = 0.07154 # Hubble's constant in Gyr^-1
        t_0 = 1 / H
        z_range = np.linspace(0, zmax, resolution)
        model_time = lambda z: 1.0 / ((1.0 + z) * self.H_z(H, self.omega_r, self.omega_m, self.omega_lambda, self.omega_o, z))
        lookBack = np.asarray([quad(model_time, 0, i)[0] for i in z_range])
        age = t_0 - np.asarray([quad(model_time, 0, i)[0] for i in z_range])
        return [z_range, lookBack, age]
    
    def calcComovingForZ(self, zmin, zmax):
        H = 2.2683e-18 # hubble constant in s^-1
        model = lambda z: 299792458.0 / self.H_z(H, self.omega_r, self.omega_m, self.omega_lambda, self.omega_o, z)
        results = quad(model, zmin, zmax)[0] 
        return results * 3.24e-23 # return results in Mpc
    
    def calcAgeForZ(self, z):
        H = 0.07154 # Hubble's constant in Gyr^-1
        t_0 = 1.0 / H
        model_time = lambda z: 1.0 / ((1.0 + z) * self.H_z(H, self.omega_r, self.omega_m, self.omega_lambda, self.omega_o, z))
        results = t_0 - quad(model_time, 0, z)[0] 
        return results
    
    def angularDist(self, zmin, zmax):
        r = self.calcComovingForZ(zmin, zmax)
        return (r / (1.0 + (zmax-zmin)))
    


# Problem 1

**Part a:**  We have three galaxies that line up where $z_1$ is the redshift from the Galaxy to galaxy 1 where as $z_2$ is from the Galaxy to galaxy 2.  That makes the difference of $z_2-z_1$ the redshift to the second galaxy as observed from galaxy 1.

To start I define the redshifts $z_1$ and $z_2$ in terms of Doppler Shift: $$z_1 = \frac{\lambda_{0,o}-\lambda_{1,e}}{\lambda_{0,o}}$$ <br/> $$z_2 = \frac{\lambda_{0,o}-\lambda_{2,e}}{\lambda_{0,o}}$$

Taking $z_2-z_1$ we get: $$z_2-z_1=  \frac{\lambda_{0,o}-\lambda_{2,e}}{\lambda_{0,o}} - \frac{\lambda_{0,o}-\lambda_{1,e}}{\lambda_{0,o}}$$
Reducing to our final answer: $$\boxed{z_2-z_1=\frac{\lambda_{1,e}-\lambda_{2,e}}{\lambda_{0,o}}}$$

**Part b:** We know from class that $d_A = \frac{S_k(r)}{1+z}$.  For a flat universe then the previous equation simplifies to the comoving distance over the redshift: $$\boxed{d_A = \frac{r}{1+z}}$$
The following code below then calculates this comoving distance for the three cases specified with $\Omega=1$, $z_1=0.5$, and $z_2=1.6$.

I should preface that to get the angular distance from galaxy 1 to 2, $z$ in the above equation will be $z_2-z_1$.



We come to find that $\boxed{d_{A,\ 0\rightarrow 1}= 1047.7\ Mpc\ }$, $\boxed{d_{A,\ 1\rightarrow 2}=800.7\ Mpc\ }$, and $\boxed{d_{A,\ 0\rightarrow 2}=1251.1\ Mpc}$

**Part c:** Clearly there is a large discrepancy in that $d_{A,\ 0\rightarrow 1}+d_{A,\ 1\rightarrow 2}\neq d_{A,\ 0\rightarrow 2}$.  I would attribute this to the fact that we can't simply add linearly the angular diameter distances to find a total distance.  Rather, angular diameter distance is dependent on the epoch of choice.

In [26]:
model_a = Cosmology(0, 1, 0)# omega_m is 1

print "Angular diameter distance from milky way to galaxy 1:", model_a.angularDist(0,0.5)
print "Angular diameter distance from milky way to galaxy 2:", model_a.angularDist(0,1.6)
print "Anuglar diameter distance from galaxy 1 to galaxy 2:", model_a.angularDist(0.5, 1.6)

Angular diameter distance from milky way to galaxy 1: 1047.72695568
Angular diameter distance from milky way to galaxy 2: 1251.14294667
Anuglar diameter distance from galaxy 1 to galaxy 2: 800.657727526


# Problem 2
We start with the idea of a cylinder being created by a galaxy moving through the Coma cluster.  The volume of this cylinder is then $V=\Sigma \sigma t$, where $\sigma t$ is the distance traveled by the galaxy.  The mean free path is defined as the distance traveled by galaxy devided by the number of galaxies $$\lambda=\frac{d}{N}= \frac{\sigma t}{N}$$
Where N is the following: $$N = n\Sigma \sigma t$$
Putting together the equations we get: $$\lambda = \frac{\sigma t}{n\Sigma\sigma t}$$
Giving: $$\boxed{\lambda = \frac{1}{\Sigma n}}$$.
Plugging in all the values with $n$ being the number density of galaxies within the half-mass radius volume gives a mean free path of $\lambda = 14.14\ Mpc$.
The mean free collision time is found with: $$t = \frac{\lambda}{\sigma}$$.
Using a conversion factor of $3.085\times 10^{13}\ km\ /\ pc$ I can easily convert to the age of when we get a collision.  The average time between collisions in the Coma cluster is then: $$\boxed{t_{coma} = 17.29\ Gyr\ }$$
This time is much larger than the Hubble time of $13.978\ Gyr$.  Similarly the collision time of the Milky Way with the Andromeda galaxy is: $$\boxed{t_{A\rightarrow MW} = 5.7\ Gyr\ }$$
This value is a fraction of what it takes to see a collision in the Coma Cluster.
